<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
import itertools
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split #--------------splitting data into test and train
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.core.pylabtools import figsize
warnings.filterwarnings("ignore")
%matplotlib inline
!sudo pip3 install keras

In [8]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import LocallyConnected1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
print(tf.__version__)
from matplotlib import pyplot as plt
from random import randint


2.8.0


##Predicción de sobrecarga en grupos de líneas de transmisión.
En esta sección se usará inferencia bayesiana para ajustar un modelo de regresión logística a datos de violación de flujo de potencia eléctrica en grupos de líneas de transmisión, que interconectan regiones eléctricas. La variable dependientes es de naturaleza binaria con un valor de uno si la línea presenta sobrecarga y cero si no. Las variables independientes son el flujo neto máximo y mínimo en la región eléctrica en un día y se calcula como la diferencia entre la demanda menos la generación en cada región.

In [9]:
df = pd.read_csv('overload.csv')
df = df.assign(const=1)
dfy = df[['L3','L5','L6','L7','L14','L15','L22','L31','L38','L39','L51','L58','L65']] #Se escoge le número de línea de transmisión
dfx = df[['CEN','NES','NOR','NTE','OCC','ORI','PEN','CEN_min','NES_min','NOR_min','NTE_min','OCC_min','ORI_min','PEN_min']] ## Predictors

X = dfx.to_numpy()
y = dfy.to_numpy()

#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()

## Crea conjuntos de datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.7) #, random_state = 5

#df.sample(5)
#df.describe()
# df.head()

A continuación se enlista las configuraciones de red usadas en el libro (para resolver el problema *11.7 Example: ZIP Code Data*). 
*   Net-1: Sin capa oculta, equivalente a regresión logística multinomial.
*   Net-2: Una capa oculta, 12 unidades ocultas totalmente conectadas.
*   Net-3: Dos capas ocultas conectadas localmente.
*   Net-4: Dos capas ocultas, conectadas localmente con peso compartido.
*   Net-5: dos capas ocultas, conectadas localmente, dos niveles de peso compartido.

Trataremos de replicar el ejercicio aplicando a nuestros datos. A diferencia del problema de 2dimensiones del libro , usaremos redes para una sola dimensión para predecir un vector de Binarios (Sobrecarga o no de algunas líneas de tranmisión) a partir de un vector de datos reales (datos de demanda de las regiones eléctricas). También mencionan que todas las redes tienen unidades de salida sigmoidales y el ajuste es con mínimpos cuadrados.


In [10]:
#https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5
#Dependencies
import keras
from   keras.models import Sequential
from   keras.layers import Dense

## Neural network
model = Sequential()
model.add(Dense(units = 26,  input_dim = 14, activation = 'sigmoid'))
#model.add(Dense(7,  input_dim = 13, activation = 'sigmoid'))
#model.add(Dense(4,  input_dim = 7,  activation = 'sigmoid'))
#model.add(Dense(14, input_dim = 4,  activation = 'sigmoid'))

model.summary()

#keras.utils.plot_model(model, "fig_t11_1.png")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 26)                390       
                                                                 
Total params: 390
Trainable params: 390
Non-trainable params: 0
_________________________________________________________________


In [11]:
#parametros https://www.tensorflow.org/api_docs/python/tf/keras/Model
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['mse'])

In [13]:
history = model.fit(X_train, y_train, epochs = 100, batch_size = 64)

Epoch 1/100
2/2 [==============================] - 2s 8ms/step - loss: 43.5522 - mse: 0.2504
Epoch 2/100
2/2 [==============================] - 0s 12ms/step - loss: 43.3635 - mse: 0.2467
Epoch 3/100
2/2 [==============================] - 0s 10ms/step - loss: 43.2538 - mse: 0.2443
Epoch 4/100
2/2 [==============================] - 0s 14ms/step - loss: 43.1620 - mse: 0.2423
Epoch 5/100
2/2 [==============================] - 0s 19ms/step - loss: 43.0815 - mse: 0.2406
Epoch 6/100
2/2 [==============================] - 0s 19ms/step - loss: 43.0103 - mse: 0.2390
Epoch 7/100
2/2 [==============================] - 0s 12ms/step - loss: 42.9416 - mse: 0.2374
Epoch 8/100
2/2 [==============================] - 0s 16ms/step - loss: 42.8828 - mse: 0.2360
Epoch 9/100
2/2 [==============================] - 0s 21ms/step - loss: 42.8306 - mse: 0.2347
Epoch 10/100
2/2 [==============================] - 0s 17ms/step - loss: 42.7734 - mse: 0.2334
Epoch 11/100
2/2 [==============================] - 0s 18ms/

In [15]:
y_pred = model.predict(X_test)
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
#Converting one hot encoded test label to label
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [16]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

Accuracy is: 3.404255319148936


In [17]:
history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=100, batch_size=64)

Epoch 1/100
2/2 [==============================] - 1s 658ms/step - loss: 43.5611 - mse: 0.1876 - val_loss: 45.5398 - val_mse: 0.1877
Epoch 2/100
2/2 [==============================] - 0s 58ms/step - loss: 43.6024 - mse: 0.1876 - val_loss: 45.6006 - val_mse: 0.1876
Epoch 3/100
2/2 [==============================] - 0s 76ms/step - loss: 43.6458 - mse: 0.1875 - val_loss: 45.6577 - val_mse: 0.1876
Epoch 4/100
2/2 [==============================] - 0s 61ms/step - loss: 43.6869 - mse: 0.1875 - val_loss: 45.7155 - val_mse: 0.1876
Epoch 5/100
2/2 [==============================] - 0s 73ms/step - loss: 43.7262 - mse: 0.1874 - val_loss: 45.7723 - val_mse: 0.1876
Epoch 6/100
2/2 [==============================] - 0s 63ms/step - loss: 43.7670 - mse: 0.1873 - val_loss: 45.8293 - val_mse: 0.1876
Epoch 7/100
2/2 [==============================] - 0s 133ms/step - loss: 43.8030 - mse: 0.1873 - val_loss: 45.8886 - val_mse: 0.1876
Epoch 8/100
2/2 [==============================] - 0s 45ms/step - loss: 43

In [18]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

KeyError: ignored

In [19]:
plt.plot(history.history['loss']) plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

SyntaxError: ignored

### **Conclusión:** 
Hemos utilizado la API funcional de **Keras** para implementar diferentes estructuras de redes neuronales para predicción de sobrecarga en líneas de transmisión de acuerdo a la demanda en las regiones.

Los resultadosindica que la mejor estructura es........... sin embargo, el ............ algunas ventajas ya que da la posibilidad de ................, mientrass.

Otra cosa que observamos es que .............., los rendimientos de predicción son similares. Esto quiere decir que a medida que crece el conjunto de datos los resultados deberían converger en la misma solución.

[Ejemplos KERAS](https://keras.io/examples/)

[Model class API](https://faroit.com/keras-docs/1.2.2/models/model/)

[The Functional API TUTORIAL](https://keras.io/guides/functional_api/)

[https://gist.github.com/jkleint/1d878d0401b28b281eb75016ed29f2ee](https://gist.github.com/jkleint/1d878d0401b28b281eb75016ed29f2ee)

[Introduction to 1D Convolutional Neural Networks in Keras for Time Sequences](https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf)

[1D Convolutional Neural Network Models for Human Activity Recognition](https://machinelearningmastery.com/cnn-models-for-human-activity-recognition-time-series-classification/)

In [ ]:
#https://stackoverflow.com/questions/49161174/tensorflow-logits-and-labels-must-have-the-same-first-dimension

#n_class = 2
#n_features = 14
#inputs = keras.Input(shape=(n_features,))

#inputs.shape
#inputs.dtype

#dense = layers.Dense(14, activation = "sigmoid")
#x = dense(inputs)

#x = layers.Dense(14, activation="sigmoid")(x)
#outputs = layers.Dense(14)(x)

#outputs = x

#model = keras.Model(inputs=inputs, outputs=outputs, name = "overload_model")

#np.expand_dims(inputs,axis=0)

#model.summary()

#keras.utils.plot_model(model, "fig_t11_1.png")
#keras.utils.plot_model(model, "fig_t11_2.png", show_shapes=True)

In [ ]:
#from keras.models import Model
#from keras.layers import Input, Dense

#a = Input(shape=(32,))
#b = Dense(32)(a)
#model = Model(Input=a, Output=b)
#compile(self, optimizer, loss, metrics=None, loss_weights=None, sample_weight_mode=None)
#fit(self, x, y, batch_size=32, nb_epoch=10, verbose=1, callbacks=None, validation_split=0.0, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)

In [ ]:
#model.compile(
#    loss      = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
#    optimizer = keras.optimizers.RMSprop(),
#    metrics=["accuracy"],
#)

#history = model.fit(x_train, y_train, batch_size=14, epochs=2, validation_split=0.2)

#test_scores = model.evaluate(x_test, y_test, verbose=2)
#print("Test loss:", test_scores[0])
#print("Test accuracy:", test_scores[1])